In [84]:
import pandas as pd
import csv as cv
import numpy as np
from ast import literal_eval
import re
from sklearn.tree import _tree
import time
import math

In [100]:
def tree_to_code(tree, feature_names, paramDict):
    f = open('TreeOutput.txt', 'w')
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    f.write("def tree({}):".format(", ".join(feature_names)))
    f.write("\n")

    def recurse(node, depth):
        indent = "  " * depth
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            f.write("{}if {} <= {}:".format(indent, name, threshold))
            f.write("\n")
            f.write("{}".format(indent)+"{")
            f.write("\n")
            
            recurse(tree_.children_left[node], depth + 1)
            f.write("{}".format(indent)+"}")
            f.write("\n")

            f.write("{}else:  # if {} > {}".format(indent, name, threshold))
            f.write("\n")

            f.write("{}".format(indent)+"{")
            f.write("\n")
            
            recurse(tree_.children_right[node], depth + 1)

            f.write("{}".format(indent)+"}")
            f.write("\n")
            
        else:
            #if paramDict['regression'] == 'yes':
            f.write("{}return {}".format(indent, tree_.value[node][0]))
            #else:
            #   f.write("{}return {}".format(indent, np.argmax(tree_.value[node][0])))
            f.write("\n")

    recurse(0, 1)
    f.close() 


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


def funcConvBranch(single_branch, dfT, rep, tree_no):
    with open('feNameType.csv') as csv_file:
        reader = cv.reader(csv_file)
        feName_type = dict(reader)
    with open('param_dict.csv') as csv_file:
        reader = cv.reader(csv_file)
        paramDict = dict(reader)

    f = open('DecSmt.smt2', 'a') 
    f.write("(assert (=> (and ")
    for i in range(0, len(single_branch)):
        temp_Str = single_branch[i]
        if 'if' in temp_Str:
            sign = re.search(r'>=|<=|>|<', temp_Str).group(0)    
            #fe_name =re.search(r'if(.*)[ sign]', temp_Str).group(1).split(sign)[0].strip()   
            fe_name = temp_Str.split('if')[1].split(sign)[0].strip()
            #print(fe_name)
            data_type = feName_type[fe_name]
            digit = float(re.search(r'[sign ][+-]?([0-9]*[.])?[0-9]+', temp_Str).group(0))
            #digit = format(digit, '.10f')
            if 'int' in data_type:
                digit = int(round(digit))
            digit = str(digit)
            f.write("(" + sign + " "+ fe_name +str(rep)+" " + digit +") ") 

        elif 'return' in temp_Str:
            digit_class = float(re.search(r'[+-]?([0-9]*[.])?[0-9]+', temp_Str).group(0))
            #digit_class = format(digit_class, '.7f')
            if paramDict['regression'] == 'no':
                digit_class = int(round(digit_class))
            digit_class = str(digit_class)
            f.write(") (= value"+str(rep)+str(tree_no)+" "+digit_class +")))")
            f.write('\n')
    f.close()


def funcGetBranch(sinBranch, dfT, rep, tree_no):
    sinBranch_set = set(sinBranch)
    return_pattern = re.compile(r'return (.*)')
    return_list = list(filter(return_pattern.match, sinBranch_set))
    if return_list:
        funcConvBranch(sinBranch, dfT, rep, tree_no)
            


def funcGenBranch(dfT, rep, paramDict, tree_no):
    with open('TreeOutput.txt') as f1:
        file_content = f1.readlines()
    file_content = [x.strip() for x in file_content]
    f1.close()
    noOfLines = file_len('TreeOutput.txt')
    temp_file_cont = ["" for x in range(noOfLines)]
    i = 1
    k = 0
    
    start_time = time.time()
    while i < noOfLines:
        j = k-1
        if temp_file_cont[j] == '}':
            funcGetBranch(temp_file_cont, dfT, rep, tree_no)
            while True:
                if temp_file_cont[j] == '{':
                    temp_file_cont[j] = ''
                    temp_file_cont[j-1] = ''
                    j = j-1
                    break  
                elif j>=0:
                    temp_file_cont[j] = ''
                    j = j-1
            k = j   
                
        else:    
            temp_file_cont[k] = file_content[i]
            k = k+1
            i = i+1
        #print('time after making a while true loop:', time.time()-start_time)    
    print('time in one loop:', time.time()-start_time)       
  
    if 'return' in file_content[1]:
        digit = float(re.search(r'[+-]?([0-9]*[.])?[0-9]+', file_content[1]).group(0))
        if paramDict['regression'] == 'no':
            digit = int(round(digit))
        f = open('DecSmt.smt2', 'a')
        f.write("(assert (= value"+str(rep)+str(tree_no)+" "+str(digit)+"))")
        f.write("\n")
        f.close()
    else:    
        funcGetBranch(temp_file_cont, dfT, rep, tree_no)


def funcConv(dfT, no_of_instances, paramDict, tree_no, total_tree):
    
    with open('feNameType.csv') as csv_file:
        reader = cv.reader(csv_file)
        feName_type = dict(reader)
    #bound_cex = literal_eval(paramDict['bound_cex'])
    
    #f.write('(set-option :pp.decimal true) \n (set-option :pp.decimal_precision 8) \n')
    if tree_no == 0:
        f = open('DecSmt.smt2', 'w')
        for j in range(0, no_of_instances):
            for i in range (0, dfT.columns.values.shape[0]-1):
                tempStr = dfT.columns.values[i]
                fe_type = feName_type[tempStr]
                min_val = dfT.iloc[:, i].min()
                max_val = dfT.iloc[:, i].max() 
            
                if 'int' in fe_type:
                    f.write("(declare-fun " + tempStr+str(j)+ " () Int)")
                    f.write('\n')
                    #adding range
                    #if bound_cex:
                    #    f.write("(assert (and (>= "+tempStr+str(j)+" "+str(min_val)+")"+" "+"(<= "+tempStr+str(j)+" "+str(max_val)+")))")
                    #f.write('\n')
                elif('float' in fe_type):
                    f.write("(declare-fun " + tempStr+str(j)+ " () Real)")
                    f.write('\n')
                    #Adding range
                    #if bound_cex:
                        #f.write("(assert (and (>= "+tempStr+str(j)+" "+str(format(min_val, '.3f'))+")"+" "+"(<= "+tempStr+str(j)+" "+str(format(max_val, '.3f'))+")))")
                     #   f.write("(assert (and (>= " + tempStr + str(j) + " " + str(0.0) + ")" + " " + "(<= " + tempStr +
                      #          str(j) + " " + str(1.0) + ")))")
                    #f.write('\n') 
                
            f.write("; "+str(j)+"th element")
            f.write('\n')
    
        for i in range(0, no_of_instances):
            for j in range(0, total_tree):
                f.write("(declare-fun value" +str(i)+str(j)+" () Real)\n")
                f.write('\n')
            f.write("(declare-fun temp_value" +str(i)+" () Real)\n")    
            if 'int' in feName_type['Class']:
                f.write("(declare-fun Class"+str(i)+" () Int)\n")
            else:
                f.write("(declare-fun Class" +str(i)+" () Real)\n")   
        #f = open('DecSmt.smt2', 'a')
        #Writing the functions for computing absolute integer & real value
        f.write('(define-fun absoluteInt ((x Int)) Int \n')
        f.write('  (ite (>= x 0) x (- x))) \n')
        f.write('(define-fun absoluteReal ((x Real)) Real \n')
        f.write('  (ite (>= x 0) x (- x))) \n')
        f.write('(declare-fun constnt () Real)\n')
        f.write('(define-fun exp ((x Real)) Real\n')
        f.write(' (^ 2.718281828459045 x))')
        f.write('(declare-fun temp () Real)\n\n\n')
        #f.write(';'+str(tree_no)+'th tree encoding\n')
        #f.close()
        
    #Calling function to get the branch and convert it to z3 form,  creating alias
    if tree_no > 0:
        f = open('DecSmt.smt2', 'a')
    f.write(';--------------'+str(tree_no)+'th tree encoding-----------------\n')
    
    for i in range(0, no_of_instances):  
        #f = open('DecSmt.smt2', 'a')
        #f.write('\n;-----------'+str(i)+'-----------number instance-------------- \n')
        f.close()
        funcGenBranch(dfT, i, paramDict, tree_no)
    

def functree2LogicMain(tree, no_of_instances, tree_no, total_tree, df):
    with open('param_dict.csv') as csv_file:
        reader = cv.reader(csv_file)
        paramDict = dict(reader)    
    tree_to_code(tree, df.columns, paramDict)
    funcConv(df, no_of_instances, paramDict, tree_no, total_tree)
    

In [101]:
from joblib import load
#gbtr = load('gtbr.joblib')
gbtr = load('gtbbin.joblib')
#df_data = pd.read_csv('TestOracleRegression.csv')
df_data = pd.read_csv('TestOracle.csv')

In [102]:
start_timey= time.time()
for i in range(0, gbtr.estimators_.shape[0]):
    functree2LogicMain(gbtr.estimators_[i, 0], 1, i, gbtr.estimators_.shape[0], df_data)
print('time in total', time.time()-start_timey)  

time in one loop: 0.0
time in one loop: 0.0
time in one loop: 0.0
time in total 0.03124213218688965


In [90]:
#For regression
learning_rate = 0.1
f1 = open('DecSmt.smt2', 'a')    
f1.write('\n;Adding the average \n')
for i in range(0, 1):
    f1.write('(assert (= Class'+str(i)+ ' (+ (* (+ ')
    for j in range(0, gbtr.estimators_.shape[0]):
        f1.write('value'+str(i)+str(j)+' ')
    f1.write(') '+str(learning_rate)+') '+str(df_data['Class'].mean())+')))\n')
    f1.write(';Encode property here\n')
    #f1.write('(assert (= Class'+str(i)+' (to_int (/ temp_class'+str(i)+' '+str(len(rf.estimators_))+')))) \n')

f1.write('(check-sat)\n')
f1.write('(get-model)')
f1.close()

In [103]:
#For binary classification
learning_rate = 0.1
f1 = open('DecSmt.smt2', 'a')    
f1.write('\n;Adding the average \n')
for i in range(0, 1):
    f1.write('(assert (= temp_value'+str(i)+' (+ (* (+ ')
    for j in range(0, gbtr.estimators_.shape[0]):
        f1.write('value'+str(i)+str(j)+' ')
    f1.write(') '+str(learning_rate)+') '+str(np.log(max(df_data['Class'].value_counts().to_dict().values())/
                                                     min(df_data['Class'].value_counts().to_dict().values())))+')))\n')
    f1.write('(assert (=> (>= temp_value'+str(i)+' 0.000001) (= Class'+str(i)+' 1)))\n')
    f1.write('(assert (=> (< temp_value'+str(i)+' 0.000001) (= Class'+str(i)+' 0)))\n')
    f1.write(';Encode property here\n')
    #f1.write('(assert (= Class'+str(i)+' (to_int (/ temp_class'+str(i)+' '+str(len(rf.estimators_))+')))) \n')

f1.write('(check-sat)\n')
f1.write('(get-model)')
f1.close()


In [82]:
np.exp(2)

7.38905609893065

In [63]:

np.log(max(df_data['Class'].value_counts().to_dict().values())/min(df_data['Class'].value_counts().to_dict().values()))

1.3862943611198906

In [86]:
math.pow(2.7182818, 2)

7.389055944211241